In [1]:
import pandas as pd
from io import StringIO
import os
import re
import numpy as np
import subprocess

In [2]:
problem_entries_path = 'problem_entries.txt'
problem_entries=open(problem_entries_path,"r").read().split('\n')

In [3]:
problem_entries

['calum', 'jamesc', 'behzad', 's0343804', 's0346429', 's0451815', 's0455730']

In [4]:
server_path = '/Volumes/Network/courses/sp/data/'

In [5]:
data_path = 'info.txt'
data_file = open(os.path.join(server_path,data_path),"r").read().split('\n')

In [6]:
data_file

['Information about the data. Only isolated English digits data are listed here.',
 '',
 '',
 'Microphone types:',
 '',
 'two types where the micophone could be at a variable distance from the speaker:',
 '"big"    : stand-mounted Sennheiser mikes (good quality)',
 '"small"  : basic Sun lapel type mic (low quality)',
 '',
 'plus various types of headset microphone:',
 '"headset" (medium quality)',
 '"logitech_headset"',
 '"sennheiser_headset"',
 '',
 'plus a variety of other microphone types, described later in this file.',
 '',
 '',
 'Where two microphone names appear (e.g. "headset/big") they are for',
 'train/test data respectively',
 '',
 'Accent types:',
 'SC : native Scottish English speaker',
 'UK : native other British English speaker',
 'AZ : native Australian/NZ speaker',
 'NA : native North American English speaker',
 'NN : non-native',
 '',
 '',
 '=================================================================',
 '',
 'There are known to be some problems with files from t

In [7]:
year_pattern = re.compile("Year*")

In [8]:
entry_pattern = re.compile("[a-z]")

In [9]:
line_pattern = re.compile('[\. a-zA-z]*:+')

In [10]:
data_dict = {}
for line in data_file:
    if year_pattern.match(line):
        year = line.split(" ")[1]
        continue
    if (entry_pattern.match(line) and (not line_pattern.match(line))):
        line = re.split(" +|\t+ *",line)
        if (len(line)>3 and len(line)<7):
            line[3:] = [' '.join(line[3:6])]
            data_dict.setdefault(year,[]).append(line)

In [11]:
data_dict.keys()

dict_keys(['2000-1', '2001-2', '2002-3', '2003-4', '2004-5', '2005-6', '2006-7', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20'])

In [12]:
tuples = []
for key in data_dict.keys():
    entry_num = len(data_dict[key])
    key_list = [key]*entry_num
    index_list = range(entry_num)
    tuples = tuples+(list(zip(key_list,index_list)))

In [13]:
index = pd.MultiIndex.from_tuples(tuples, names=['Year','Index'])

In [14]:
info_df = pd.DataFrame(columns=['Name','Gender','Microphone','Accent'])
for key in data_dict.keys():
    year_df = pd.DataFrame(data_dict[key],columns=['Name','Gender','Microphone','Accent'])
    info_df=info_df.append(year_df)

In [15]:
info_df.index=index

In [16]:
info_df.shape

(503, 4)

In [17]:
gender = info_df.loc['2013-14']['Microphone'].copy(True)
mic = info_df.loc['2013-14']['Accent'].copy(True)
accent = info_df.loc['2013-14']['Gender'].copy(True)
info_df.loc['2013-14']['Microphone']=mic
info_df.loc['2013-14']['Gender']=gender
info_df.loc['2013-14']['Accent']=accent

In [18]:
info_df.loc['2013-14']

,Name,Gender,Microphone,Accent
Index,,,,
0,s1312082,Male,logitech_headset Praat Lab,NN
1,s1355318,Female,logitech_headset Audacity Lab,NN
2,s1367766,Female,logitech_headset Praat Lab,NN
3,s1038946,Male,Apple_EarPods Audacity Lab,NN
4,s0950147,Female,iMac_inbuilt Audacity Lab,UK
5,s1336117,Female,iMac_inbuilt Audacity Lab,NN
6,s1036740,Female,logitech_headset Praat Lab,NN
7,s1114083,Female,Acer_v5-571_inbuilt Audacity Home,NA
8,s0927462,Female,iMac_inbuilt Audacity Lab,UK


In [19]:
for p in problem_entries:
    indexName = info_df[info_df['Name']==p].index
    info_df.drop(indexName,inplace=True)

In [20]:
info_df.to_csv('recording_info.csv')

## Specify the selecting requirements

In [21]:
selection = info_df[~info_df['Gender'].isin(['f','Female'])]['Name'].values

In [22]:
selection

array(['chongh', 'jamesh', 'markm', 'scottn', 'simonk', 'alex',
       'anthonyb', 'benjamin', 'ccb', 'christop', 'colinb', 'davidr',
       'yokong', 'athan', 'davidt', 'guy', 'jose', 'michaelg', 'michaels',
       'ptepper', 'spengler', 'timm', 'wouter', 's0345201', 's9901729',
       's0345701', 's0340933', 's0348129', 's0344328', 's0343559',
       's0345347', 's0343444', 'joshua', 's0343879', 's0343799',
       's0345336', 's0349492', 's9810638', 's0126339', 's0450736',
       's0453481', 's0454291', 's0455511', 's0457086', 'taal11',
       's0235479', 's0565563', 's0451342', 's0672048', 's0677387',
       's0678004', 's0680940', 's0681481', 's0836504', 's1052161',
       's1052483', 's1054017', 's9061219', 's0680570', 's0806577',
       's0824968', 's0832212', 's0833088', 's0927722', 's0940658',
       's1100218', 's1125835', 's1128289', 's1135029', 's1151467',
       's1157551', 's0906559', 's0907414', 's0949139', 's1021894',
       's1200359', 's1250520', 's1261038', 's1312082'

Generate the files for HTK

In [23]:
# Function for writing the List_inti_train list
# `names` is a list of user names selected before
# `path` is the path to the scp file that we want to write to
def prepareInit(names,path):
    print("Preparing training data list...\n")
    with open(path,'w+') as init:
        for name in names:
            print(name)
            init.write("/Volumes/Network/courses/sp/data/mfcc/train/{}_train.mfcc\n".format(name))
    init.close()
    print("Done\n")
def prepareRecog(names,path):
    print("Preparing testing data list...\n")
    with open(path,'w+') as rec:
        for name in names:
            print(name)
            rec.write("/Volumes/Network/courses/sp/data/mfcc/test/{}_test*.mfcc\n".format(name))
    rec.close()
    print("Done\n")
def prepare_res_label(names,path):
    print("Preparing testing master files list...\n")
    with open(path,'w+') as lab:
        for name in names:
            lab.write("/Volumes/Network/courses/sp/data/lab/test/{}_test.mlf\n".format(name))
            print(name)
    lab.close()
    print("Done\n")
def prepare_res_rec(names,recogresult_path,list_path):
    print("Preparing testing recognition results...\n")
    file_names = [rec for rec in os.listdir(recogresult_path) if rec.split('_')[0] in names and len(rec.split('_'))<3 and re.match(('[\w]*test(?=[0-9])'),rec)]
    with open(list_path,'w+') as rec_result:
        for file_name in file_names:
            rec_result.write("../rec/{}\n".format(file_name))
            print(file_name)
    rec_result.close()
    print("Done\n")

In [24]:
# The path for all lists and specific lists
# Path to the directory containing all the recognition results
recogresult = "../rec/"
# Path to the lists for traininig and testing
listpath = "../list/"
initpath = os.path.join(listpath,"list_init_train.scp")
recopath = os.path.join(listpath,"list_recogniser.scp")
labepath = os.path.join(listpath,"list_results_label.scp")
r_repath = os.path.join(listpath,"list_results_rec.scp")

## Experiment 1 | test with a five users' data

In [25]:

def run_experiment(trn_list,tst_list):
    prepareInit(trn_list,initpath)
    prepareRecog(trn_list,recopath)
    subprocess.call('../scripts/mmulti_init_train')
    prepare_res_label(trn_list,labepath)
    subprocess.call('../scripts/multi_recognise_test_data')
    prepare_res_rec(trn_list,recogresult,r_repath)
    result = None
    try:
        result=subprocess.check_output('../scripts/multi_results')        
        result=result.decode("utf-8")
        if(not result):
            print("empty result")
            return None
    except:
        print("bad user {}".format(trn_list))
        return None
    return result
#     result=subprocess.check_output('../scripts/mmulti_recognise_result')
#     print(result)

In [26]:
def parse_result(result):    
    m = re.search('Corr=(.+?),', result)
    if m:
        found = m.group(1)
        print(found)
    return float(found)

## DONE

## Experient 2 | Remove users with WER higher than 0%

In [27]:
all_users = info_df['Name'].values;all_users

array(['nicolek', 'chongh', 'jamesh', 'markm', 'scottn', 'simonk',
       'annika', 'beatrice', 'elisabet', 'fionac', 'michele', 'nicola',
       'panagiot', 'sarah', 'sasha', 'alex', 'anthonyb', 'benjamin',
       'ccb', 'christop', 'colinb', 'davidr', 'yokong', 'andra', 'angie',
       'beata', 'efi', 'kateb', 'kelly', 'kit', 'kristina', 'mariza',
       'milena', 'rachel', 'stav', 'unni', 'vwiersma', 'yoko', 'athan',
       'davidt', 'guy', 'jose', 'michaelg', 'michaels', 'ptepper',
       'spengler', 'timm', 'wouter', 's0345201', 's0343373', 's9901729',
       's0345701', 's0345310', 's0340933', 's0341098', 's0348129',
       's0343949', 's0344328', 's0343559', 's0345347', 's0343820',
       's0343444', 'joshua', 's0343879', 's0343799', 's0348176',
       's0349395', 's0345336', 's0349492', 's0339733', 's9810638',
       's0340681', 's0125605', 's0126339', 's0127196', 's0231469',
       's0450736', 's0450918', 's0450928', 's0450990', 's0453481',
       's0453738', 's0453767', 's045

In [28]:
run_experiment(['panagiot'],['panagiot'])

Preparing training data list...

panagiot
Done

Preparing testing data list...

panagiot
Done

Preparing testing master files list...

panagiot
Done

Preparing testing recognition results...

panagiot_test08.rec
panagiot_test09.rec
panagiot_test30.rec
panagiot_test25.rec
panagiot_test10.rec
panagiot_test04.rec
panagiot_test21.rec
panagiot_test05.rec
panagiot_test26.rec
panagiot_test17.rec
panagiot_test19.rec
panagiot_test27.rec
panagiot_test01.rec
panagiot_test11.rec
panagiot_test12.rec
panagiot_test06.rec
panagiot_test29.rec
panagiot_test15.rec
panagiot_test14.rec
panagiot_test18.rec
panagiot_test22.rec
panagiot_test02.rec
panagiot_test07.rec
panagiot_test24.rec
panagiot_test16.rec
panagiot_test23.rec
panagiot_test03.rec
panagiot_test20.rec
panagiot_test13.rec
panagiot_test28.rec
Done



'WORD: %Corr=36.67, Acc=36.67 [H=11, D=0, S=19, I=0, N=30]\n'

In [29]:
bad_users = []
for user in all_users:
    result = run_experiment([user],[user])
    
    if result==None:
        bad_users.append(user)
        continue
    if parse_result(result) < 100:
        bad_users.append(user)
        continue
bad_users

Preparing training data list...

nicolek
Done

Preparing testing data list...

nicolek
Done

Preparing testing master files list...

nicolek
Done

Preparing testing recognition results...

nicolek_test03.rec
nicolek_test23.rec
nicolek_test07.rec
nicolek_test19.rec
nicolek_test11.rec
nicolek_test02.rec
nicolek_test30.rec
nicolek_test14.rec
nicolek_test26.rec
nicolek_test08.rec
nicolek_test01.rec
nicolek_test27.rec
nicolek_test15.rec
nicolek_test16.rec
nicolek_test25.rec
nicolek_test05.rec
nicolek_test10.rec
nicolek_test04.rec
nicolek_test09.rec
nicolek_test12.rec
nicolek_test29.rec
nicolek_test18.rec
nicolek_test28.rec
nicolek_test20.rec
nicolek_test17.rec
nicolek_test13.rec
nicolek_test06.rec
nicolek_test21.rec
nicolek_test24.rec
nicolek_test22.rec
Done

83.33
Preparing training data list...

chongh
Done

Preparing testing data list...

chongh
Done

Preparing testing master files list...

chongh
Done

Preparing testing recognition results...

chongh_test20.rec
chongh_test01.rec
chongh_

nicola
Done

Preparing testing data list...

nicola
Done

Preparing testing master files list...

nicola
Done

Preparing testing recognition results...

nicola_test29.rec
nicola_test20.rec
nicola_test07.rec
nicola_test16.rec
nicola_test11.rec
nicola_test21.rec
nicola_test18.rec
nicola_test28.rec
nicola_test10.rec
nicola_test19.rec
nicola_test25.rec
nicola_test13.rec
nicola_test03.rec
nicola_test08.rec
nicola_test02.rec
nicola_test22.rec
nicola_test04.rec
nicola_test27.rec
nicola_test06.rec
nicola_test23.rec
nicola_test09.rec
nicola_test24.rec
nicola_test05.rec
nicola_test12.rec
nicola_test14.rec
nicola_test17.rec
nicola_test30.rec
nicola_test26.rec
nicola_test15.rec
nicola_test01.rec
Done

96.67
Preparing training data list...

panagiot
Done

Preparing testing data list...

panagiot
Done

Preparing testing master files list...

panagiot
Done

Preparing testing recognition results...

panagiot_test08.rec
panagiot_test09.rec
panagiot_test30.rec
panagiot_test25.rec
panagiot_test10.rec
pan

66.67
Preparing training data list...

yokong
Done

Preparing testing data list...

yokong
Done

Preparing testing master files list...

yokong
Done

Preparing testing recognition results...

yokong_test13.rec
yokong_test18.rec
yokong_test09.rec
yokong_test27.rec
yokong_test22.rec
yokong_test21.rec
yokong_test05.rec
yokong_test02.rec
yokong_test08.rec
yokong_test14.rec
yokong_test10.rec
yokong_test06.rec
yokong_test30.rec
yokong_test04.rec
yokong_test03.rec
yokong_test26.rec
yokong_test07.rec
yokong_test23.rec
yokong_test20.rec
yokong_test19.rec
yokong_test11.rec
yokong_test15.rec
yokong_test24.rec
yokong_test17.rec
yokong_test25.rec
yokong_test28.rec
yokong_test12.rec
yokong_test29.rec
yokong_test16.rec
yokong_test01.rec
Done

100.00
Preparing training data list...

andra
Done

Preparing testing data list...

andra
Done

Preparing testing master files list...

andra
Done

Preparing testing recognition results...

andra_test23.rec
andra_test03.rec
andra_test19.rec
andra_test01.rec
andr

Done

100.00
Preparing training data list...

stav
Done

Preparing testing data list...

stav
Done

Preparing testing master files list...

stav
Done

Preparing testing recognition results...

stav_test24.rec
stav_test01.rec
stav_test20.rec
stav_test15.rec
stav_test14.rec
stav_test23.rec
stav_test17.rec
stav_test27.rec
stav_test08.rec
stav_test11.rec
stav_test19.rec
stav_test02.rec
stav_test18.rec
stav_test30.rec
stav_test03.rec
stav_test13.rec
stav_test12.rec
stav_test16.rec
stav_test10.rec
stav_test26.rec
stav_test29.rec
stav_test28.rec
stav_test04.rec
stav_test21.rec
stav_test07.rec
stav_test06.rec
stav_test05.rec
stav_test25.rec
stav_test09.rec
stav_test22.rec
Done

80.00
Preparing training data list...

unni
Done

Preparing testing data list...

unni
Done

Preparing testing master files list...

unni
Done

Preparing testing recognition results...

unni_test21.rec
unni_test05.rec
unni_test22.rec
unni_test01.rec
unni_test17.rec
unni_test09.rec
unni_test14.rec
unni_test24.rec
unni_te

Done

96.67
Preparing training data list...

timm
Done

Preparing testing data list...

timm
Done

Preparing testing master files list...

timm
Done

Preparing testing recognition results...

timm_test04.rec
timm_test22.rec
timm_test24.rec
timm_test18.rec
timm_test23.rec
timm_test26.rec
timm_test20.rec
timm_test16.rec
timm_test06.rec
timm_test07.rec
timm_test03.rec
timm_test13.rec
timm_test30.rec
timm_test19.rec
timm_test09.rec
timm_test28.rec
timm_test21.rec
timm_test11.rec
timm_test12.rec
timm_test02.rec
timm_test25.rec
timm_test10.rec
timm_test15.rec
timm_test29.rec
timm_test14.rec
timm_test05.rec
timm_test17.rec
timm_test01.rec
timm_test08.rec
timm_test27.rec
Done

100.00
Preparing training data list...

wouter
Done

Preparing testing data list...

wouter
Done

Preparing testing master files list...

wouter
Done

Preparing testing recognition results...

wouter_test23.rec
wouter_test30.rec
wouter_test25.rec
wouter_test14.rec
wouter_test24.rec
wouter_test09.rec
wouter_test04.rec
wou

...

Preparing testing master files list...

s1941395
Done

Preparing testing recognition results...

s1941395_test14.rec
s1941395_test06.rec
s1941395_test21.rec
s1941395_test26.rec
s1941395_test09.rec
s1941395_test17.rec
s1941395_test05.rec
s1941395_test08.rec
s1941395_test04.rec
s1941395_test30.rec
s1941395_test10.rec
s1941395_test02.rec
s1941395_test16.rec
s1941395_test23.rec
s1941395_test19.rec
s1941395_test11.rec
s1941395_test12.rec
s1941395_test07.rec
s1941395_test18.rec
s1941395_test03.rec
s1941395_test20.rec
s1941395_test27.rec
s1941395_test01.rec
s1941395_test24.rec
s1941395_test13.rec
s1941395_test22.rec
s1941395_test15.rec
s1941395_test29.rec
s1941395_test25.rec
s1941395_test28.rec
Done

100.00
Preparing training data list...

s1916940
Done

Preparing testing data list...

s1916940
Done

Preparing testing master files list...

s1916940
Done

Preparing testing recognition results...

s1916940_test15.rec
s1916940_test05.rec
s1916940_test01.rec
s1916940_test14.rec
s1916940_test27.re

Preparing testing master files list...

s1988523
Done

Preparing testing recognition results...

s1988523_test13.rec
s1988523_test01.rec
s1988523_test14.rec
s1988523_test06.rec
s1988523_test29.rec
s1988523_test05.rec
s1988523_test09.rec
s1988523_test20.rec
s1988523_test03.rec
s1988523_test08.rec
s1988523_test28.rec
s1988523_test27.rec
s1988523_test23.rec
s1988523_test07.rec
s1988523_test10.rec
s1988523_test30.rec
s1988523_test12.rec
s1988523_test26.rec
s1988523_test16.rec
s1988523_test22.rec
s1988523_test25.rec
s1988523_test11.rec
s1988523_test24.rec
s1988523_test17.rec
s1988523_test02.rec
s1988523_test18.rec
s1988523_test04.rec
s1988523_test21.rec
s1988523_test19.rec
s1988523_test15.rec
Done

100.00
Preparing training data list...

s2000148
Done

Preparing testing data list...

s2000148
Done

Preparing testing master files list...

s2000148
Done

Preparing testing recognition results...

s2000148_test29.rec
s2000148_test14.rec
s2000148_test01.rec
s2000148_test24.rec
s2000148_test20.re

Preparing testing master files list...

s1764494
Done

Preparing testing recognition results...

s1764494_test07.rec
s1764494_test06.rec
s1764494_test21.rec
s1764494_test20.rec
s1764494_test08.rec
s1764494_test09.rec
s1764494_test30.rec
s1764494_test10.rec
s1764494_test26.rec
s1764494_test22.rec
s1764494_test13.rec
s1764494_test11.rec
s1764494_test15.rec
s1764494_test24.rec
s1764494_test04.rec
s1764494_test02.rec
s1764494_test29.rec
s1764494_test16.rec
s1764494_test01.rec
s1764494_test28.rec
s1764494_test17.rec
s1764494_test19.rec
s1764494_test12.rec
s1764494_test25.rec
s1764494_test23.rec
s1764494_test05.rec
s1764494_test18.rec
s1764494_test27.rec
s1764494_test14.rec
s1764494_test03.rec
Done

86.67
Preparing training data list...

s1735938
Done

Preparing testing data list...

s1735938
Done

Preparing testing master files list...

s1735938
Done

Preparing testing recognition results...

s1735938_test12.rec
s1735938_test01.rec
s1735938_test02.rec
s1735938_test18.rec
s1735938_test29.rec

Preparing testing master files list...

s1940488
Done

Preparing testing recognition results...

s1940488_test15.rec
s1940488_test19.rec
s1940488_test14.rec
s1940488_test10.rec
s1940488_test18.rec
s1940488_test03.rec
s1940488_test08.rec
s1940488_test04.rec
s1940488_test29.rec
s1940488_test28.rec
s1940488_test05.rec
s1940488_test30.rec
s1940488_test13.rec
s1940488_test02.rec
s1940488_test21.rec
s1940488_test12.rec
s1940488_test07.rec
s1940488_test25.rec
s1940488_test23.rec
s1940488_test20.rec
s1940488_test01.rec
s1940488_test09.rec
s1940488_test16.rec
s1940488_test27.rec
s1940488_test26.rec
s1940488_test24.rec
s1940488_test17.rec
s1940488_test06.rec
s1940488_test11.rec
s1940488_test22.rec
Done

96.67
Preparing training data list...

s1306161
Done

Preparing testing data list...

s1306161
Done

Preparing testing master files list...

s1306161
Done

Preparing testing recognition results...

s1306161_test13.rec
s1306161_test08.rec
s1306161_test14.rec
s1306161_test16.rec
s1306161_test23.rec

['nicolek',
 'chongh',
 'markm',
 'beatrice',
 'elisabet',
 'fionac',
 'michele',
 'nicola',
 'panagiot',
 'sarah',
 'sasha',
 'alex',
 'anthonyb',
 'benjamin',
 'christop',
 'colinb',
 'davidr',
 'andra',
 'angie',
 'efi',
 'kateb',
 'kelly',
 'kit',
 'kristina',
 'mariza',
 'milena',
 'stav',
 'unni',
 'vwiersma',
 'yoko',
 'athan',
 'guy',
 'jose',
 'michaelg',
 'ptepper',
 'spengler',
 'wouter',
 's0345201',
 's0343373',
 's9901729',
 's0345701',
 's0345310',
 's0340933',
 's0341098',
 's0345347',
 's0343820',
 's0343444',
 's0343879',
 's0343799',
 's0348176',
 's0349492',
 's0339733',
 's9810638',
 's0340681',
 's0125605',
 's0231469',
 's0450736',
 's0450928',
 's0450990',
 's0453481',
 's0453767',
 's0454417',
 's0454931',
 's0454934',
 's0457086',
 's0235479',
 's0560804',
 's0562500',
 's0563087',
 's0565890',
 's0568074',
 's0570475',
 's0238734',
 's0452991',
 's0672048',
 's0674231',
 's0674876',
 's0678004',
 's0680940',
 's0681481',
 's0815365',
 's0830024',
 's0834892',

In [30]:
len(bad_users)#after modification, should be less bad users

245

In [184]:
len(bad_users)

256

In [31]:
info_df.shape

(498, 4)

In [32]:
for p in bad_users:
    indexName = info_df[info_df['Name']==p].index
    info_df.drop(indexName,inplace=True)

In [33]:
info_df.to_csv('checked_recording_info.csv')